In [1]:
# -- IMPORTS -- #
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, InputLayer, Dense, Conv2D, MaxPooling2D, Flatten, Activation
from keras.models import Sequential, Model
import keras.activations as kerasAct
import PIL.Image as pilImage
import keras.backend as K
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
# -- DEFINE DECONV LAYER CLASSES -- #
# -- CONV2D DECONOLUTION -- #
class DConv2D(object):
    # -- INITIALIZATION -- #
    def __init__(self,layer):
        self.layer=layer
        weights = layer.get_weights()
        W = weights[0]; b = weights[1]
        # -- UP FUNCTION -- #
        upFilter = W.shape[0]
        upRow = W.shape[2]
        upCol = W.shape[3]
        layerInput = Input(shape = layer.input_shape[1:])
        layerOutput = Conv2D(nb_filter = upFilter,
                             nb_row = row,
                             nb_col = col, 
                             border_mode = 'same',
                             weights = [W,b])(layerInput)
        self.up_function = K.function([layerInput],[layerOutput])
        # -- DOWN FUNCTION -- #
        W = np.transpose(W,(1,0,2,3))
        W = W[:,:,::-1,::-1]
        downFilter = W.shape[0]
        downRow = W.shape[2]
        downCol = W.shape[3]
        b = np.zeros(downFilter)
        inputLayer = Input(shape = layer.output_shape[1:])
        outputLayer = Conv2D(nb_filter = downFilter,
                             nb_row = downRow,
                             nb_col = downCol,
                             border_mode = 'same',
                             weights = [W,b])(inputLayer)
        self.down_function = K.function([inputLayer],[outputLayer])
    # -- UP PASS -- #
    def up(self,data):
        self.upData = self.up_function([data])
        return self.upData
    # -- DOWN PASS -- #
    def down(self,data):
        self.downData = self.down_function([data])
        return self.downData
    
# -- DENSE DECONVOLUTION -- #
class DDense(object):
    def __init__(self,layer):
        self.layer = layer
        weights = layer.get_weights()
        W = weights[0]
        b = weights[1]
        # -- UP FUNCTION -- #
        inputLayer = Input(shape = layer.input_shape[1:])
        outputLayer = Dense(output_dim = layer.output_shape[1],weights=[W,b])(inputLayer)
        slef.up_function = K.function([inputLayer],[outputLayer])
        # -- DOWN FUNCTION -- #
        W = W.transpose()
        b = np.zeros(layer.input_shape[1])
        inputLayer = Input(shape = layer.output_shape[1:])
        outputLayer = Dense(output_dim = layer.input_shape[1],
                            weights = [W,b])(inputLayer)
        self.down_function = K.function([inputLayer],[outputLayer])
    # -- UP PASS -- #
    def up(self,data):
        self.upData = self.up_function([data])
        return self.upData
    # -- DOWN PASS -- #
    def down(self,data):
        self.downData = self.down_function([data])
        return self.downData
    
# -- POOLING LAYERS DECONVOLUTION -- #
class DPooling(object):
    def __init__(self,layer):
        self.layer = layer
        self.poolSize = layer.pool_size
    # -- UP PASS -- #
    def up(self,data):
        [self.upData,self.switch] = self._max_pooling_with_switch(data,self.poolSize)
        return self.upData
    # -- DOWN PASS -- #
    def down(self,data):
        self.downData = self._max_unpooling_with_switch(data,self.switch)
        return self.downData
    # -- FORWARD POOLING -- #
    def _max_pooling_with_switch(self,data,poolSize):
        switch = np.zeros(data.shape)
        outShape = list(data.shape)
        outShape[2] = outShape[2]/poolSize[0]
        outShape[3] = outShape[3]/poolSize[1]
        rowPool = int(poolSize[0])
        colPool = int(poolSize[1])
        pooled = np.zeros(outShape)
        for sample in range(data.shape[0]):
            for dim in range(data.shape[1]):
                for row in range(outShape[2]):
                    for col in range(outShape[3]):
                        patch = data[sample,
                                     dim,
                                     row*rowPool : (row+1)*rowPool,
                                     col*colPool:(col+1)*colPool]
                        maxVal = np.max(patch)
                        pooled[sample,dim,row,col] = maxVal
                        maxRow = np.argmax(np.max(patch,axis=1))
                        maxCol = np.argmax(patch,axis=1)[maxRow]
                        switch[sample,
                               dim,
                               row*rowPool+maxRow,
                               col*colPool+maxCol] = 1
        return [pooled,switch]
    # -- BACKWARD POOLING -- #
    def _max_unpooling_with_switch(self,data,switch):
        tile = np.ones((switch.shape[2]/data.shape[2], switch.shape[3]/data.shape[3]))
        return np.kron(data,tile)*switch

# -- ACTIVATIONS DECONVOLUTIONS -- #
class DActivation(object):
    def __init__(self,layer,linear=False):
        self.layer = layer
        self.linear = linear
        slef.activation = layer.activation
        inputLayer = K.placeholder(shape = layer.output_shape)
        outputLayer = self.activation(inputLayer)
        # -- RELU ACTIVATION -- #
        self.up_function = K.function([inputLayer],[outputLayer])
        slef.down_function = K.function([inputLayer],[outputLayer])
    def up(self,data):
        self.upData = self.up_function([data])
        return self.upData
    def down(self,data):
        self.downData = self.down_function([data])
        return self.downData
    
# -- FLATTEN DECONVOLUTION -- #
class DFlatten(object):
    def __init__(self,layer):
        self.layer = layer
        self.shape = layer.input_shape[1:]
        self.up_function = K.function([layer.input],[layer.output])
    def up(self,data):
        self.upData = self.up_function([data])
        return self.upData
    def down(self,data):
        shape = [data.shape[0]]+list(self.shape)
        assert np.prod(self.shape) == np.prod(data.shape[1:])
        self.downData = np.reshape(data,shape)
        return self.downData
    
# -- INPUT DECONVOLUTION -- #
class DInput(object):
    def __init__(self,layer):
        self.layer = layer
    def up(self,data):
        self.upData = data
        return self.upData
    def down(self,data):
        self.downData = data
        return self.downData
    
# -- BATCH NORMALIZATION DECONVOLUTION -- #


In [ ]:
# -- VISUALIZATION FUNCTION -- #
def visualize(model,data,layerName,feature,visualizeMode):
    

In [6]:
model = VGG16(weights='imagenet',include_top=True)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
l = model.get_layer('block3_conv3').get_weights()
W = l[0]
print(W.shape)

(3, 3, 256, 256)
